In [275]:
import pandas as pd
# json to pandas dataframe

In [ ]:
!pip install spotipy

In [288]:
nothotsongs=pd.read_csv('not_hot_songs.csv', encoding='latin1')
nothotsongs.head()
#not_hot_song=nothotsongs.loc[:,['title','artist']]
#not_hot_song.head()

,Unnamed: 0,title,artist
0,0,Like a Rolling Stone,Bob Dylan
1,1,(I Can't Get No) Satisfaction',The Rolling Stones
2,2,Imagine,John Lennon
3,3,What's Going On,Marvin Gaye
4,4,Respect,Aretha Franklin


In [287]:
hotsongs=pd.read_csv('hotsongs.csv', encoding='latin1')
hotsongs.head()

,Unnamed: 0,title,artist
0,0,Anti-Hero,Taylor Swift
1,1,Lift Me Up,Rihanna
2,2,Unholy,Sam Smith & Kim Petras
3,3,Bad Habit,Steve Lacy
4,4,As It Was,Harry Styles


## At this point, you have the hot_songs and the not_hot_songs databases. However, you don't have any acoustic information about the songs. The purpose of this lab is to use Spotify's API to extend both databases with this information for each song included to use this information later.

In [278]:
from config import *

Create a function to search a given single song in the Spotify API: search_song(title, artist).
Take into account that sometimes Spotify's API will return several matches for the same song title (different artists, a different album of the same artist, version of the song,...etc). Then it will be nice to display a list of outputs to the user and let him/her select which is the right match. Once the desired song is located, the function should return the href/id/uri of the song to the code (not to the user). Keep in mind, that a given song might not be available on Spotify's API (make sure to use the song's title and artist searching the song). If the song is not found, the function must return an empty string as the href/id/uri. Also, in this case, you should remove this song from the database.

In [304]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= Client_ID,
                                                           client_secret= Client_Secret))


In [280]:
def asking_id(df):
        
    df2 = df.copy()
    song_ids = []
    for i in range(0,len(df2),50):
        print("Chunk: ",i)
        titles = list(df2['title'][i:i+50].values)
        artists = list(df2['artist'][i:i+50].values)
        elems = list(zip(titles,artists))
        for item in elems:
            string = "track:" + item[0] + " artist:" + item[1]
            print("Searching for song: ",item[0],"from artist: ", item[1])
            try:
                results = sp.search(string, limit=1)
                song_id = results['tracks']['items'][0]['id']
                song_ids.append(song_id)
            except:
                song_id = "None"
                song_ids.append(song_id)
                print("Song not found!")
        print()
        print("Waiting 10 seconds before the next chunk")
        sleep(10)

    df2['id'] = song_ids
    
    return df2
        
        

In [281]:
from time import sleep #we start with the hotsongs



## Create a function get_audio_features(list_of_songs) to obtain the audio features of a givenlist of songs (the content of list_of_songs can be the href/id/uri).

In [295]:
def asking_features(df):
    
    df1=pd.DataFrame()
    
    df3 = df.copy()
    
    for i in range(0,len(df3),50):
        print("Chunk: ",i)
        elems = list(df3['id'][i:i+50].values) #this is my list of ids 
        for item in elems:
            if item != 'None':
                features = sp.audio_features((item))[0]
                my_dict_new = { key : [features[key]] for key in list(features.keys()) }
                new_df=pd.DataFrame(my_dict_new)
                
                df1 = pd.concat([df1, new_df], axis=0) 
          
            else:
        
                print('uri is None')
        print("Waiting 10 seconds before the next chunk")
        sleep(10)

  
    return df3, df1


In [296]:
result  = asking_id(hotsongs)
result = result.loc[~result['id'].str.contains('None')]
df_titles, df_features_hot_song=asking_features(result)

Chunk:  0
Searching for song:  Anti-Hero from artist:  Taylor Swift
Searching for song:  Lift Me Up from artist:  Rihanna
Searching for song:  Unholy from artist:  Sam Smith & Kim Petras
Song not found!
Searching for song:  Bad Habit from artist:  Steve Lacy
Searching for song:  As It Was from artist:  Harry Styles
Searching for song:  Lavender Haze from artist:  Taylor Swift
Searching for song:  Midnight Rain from artist:  Taylor Swift
Searching for song:  I Like You (A Happier Song) from artist:  Post Malone Featuring Doja Cat
Song not found!
Searching for song:  Bejeweled from artist:  Taylor Swift
Searching for song:  Super Freaky Girl from artist:  Nicki Minaj
Searching for song:  Shirt from artist:  SZA
Searching for song:  Maroon from artist:  Taylor Swift
Searching for song:  I Ain't Worried from artist:  OneRepublic
Searching for song:  You Proof from artist:  Morgan Wallen
Searching for song:  I'm Good (Blue) from artist:  David Guetta & Bebe Rexha
Song not found!
Searching f

In [297]:
#here I print out the df after use of function with features, first part has titles and id
df_titles 


,Unnamed: 0,title,artist,id
0,0,Anti-Hero,Taylor Swift,0V3wPSX9ygBnCm8psDIegu
1,1,Lift Me Up,Rihanna,35ovElsgyAtQwYPYnZJECg
3,3,Bad Habit,Steve Lacy,5CM4UuQ9Gnd6K2YyKGPMoK
4,4,As It Was,Harry Styles,4LRPiXqCikLlN15c3yImP7
5,5,Lavender Haze,Taylor Swift,5jQI2r1RdgtuT8S3iG8zFC
...,...,...,...,...
93,93,Betty (Get Money),Yung Gravy,4cacyP5c3PMlfnyjpg13xW
95,95,300 Blackout,Kodak Black,3y3NwQPQAjKCunRz2eKggA
96,96,Dear Reader,Taylor Swift,3QF5RsWzK1lCvf2o2cY65P
97,97,Country On,Luke Bryan,1tRxwf8Q0AcshfHuaD86Yt


In [298]:
#this is the df with the features
df_features_hot_song

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.637,0.643,4,-6.571,1,0.0519,0.13000,0.000002,0.1420,0.5330,97.008,audio_features,0V3wPSX9ygBnCm8psDIegu,spotify:track:0V3wPSX9ygBnCm8psDIegu,https://api.spotify.com/v1/tracks/0V3wPSX9ygBn...,https://api.spotify.com/v1/audio-analysis/0V3w...,200690,4
0,0.247,0.299,9,-6.083,1,0.0315,0.89900,0.000000,0.1310,0.1720,177.115,audio_features,35ovElsgyAtQwYPYnZJECg,spotify:track:35ovElsgyAtQwYPYnZJECg,https://api.spotify.com/v1/tracks/35ovElsgyAtQ...,https://api.spotify.com/v1/audio-analysis/35ov...,196520,4
0,0.686,0.507,1,-7.097,1,0.0357,0.62600,0.000075,0.3830,0.6900,168.955,audio_features,5CM4UuQ9Gnd6K2YyKGPMoK,spotify:track:5CM4UuQ9Gnd6K2YyKGPMoK,https://api.spotify.com/v1/tracks/5CM4UuQ9Gnd6...,https://api.spotify.com/v1/audio-analysis/5CM4...,232115,4
0,0.520,0.731,6,-5.338,0,0.0557,0.34200,0.001010,0.3110,0.6620,173.930,audio_features,4LRPiXqCikLlN15c3yImP7,spotify:track:4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303,4
0,0.733,0.436,10,-10.489,1,0.0800,0.25800,0.000573,0.1570,0.0976,96.985,audio_features,5jQI2r1RdgtuT8S3iG8zFC,spotify:track:5jQI2r1RdgtuT8S3iG8zFC,https://api.spotify.com/v1/tracks/5jQI2r1Rdgtu...,https://api.spotify.com/v1/audio-analysis/5jQI...,202396,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.723,0.772,1,-6.784,1,0.0792,0.00524,0.000012,0.3670,0.5800,102.004,audio_features,4cacyP5c3PMlfnyjpg13xW,spotify:track:4cacyP5c3PMlfnyjpg13xW,https://api.spotify.com/v1/tracks/4cacyP5c3PMl...,https://api.spotify.com/v1/audio-analysis/4cac...,146471,4
0,0.669,0.741,0,-6.545,0,0.3160,0.01980,0.000000,0.2230,0.3960,167.903,audio_features,3y3NwQPQAjKCunRz2eKggA,spotify:track:3y3NwQPQAjKCunRz2eKggA,https://api.spotify.com/v1/tracks/3y3NwQPQAjKC...,https://api.spotify.com/v1/audio-analysis/3y3N...,166494,4
0,0.562,0.388,0,-12.088,1,0.0705,0.48100,0.001450,0.1170,0.1590,107.747,audio_features,3QF5RsWzK1lCvf2o2cY65P,spotify:track:3QF5RsWzK1lCvf2o2cY65P,https://api.spotify.com/v1/tracks/3QF5RsWzK1lC...,https://api.spotify.com/v1/audio-analysis/3QF5...,225194,4
0,0.520,0.751,5,-5.064,1,0.0551,0.17100,0.000007,0.0624,0.5190,156.044,audio_features,1tRxwf8Q0AcshfHuaD86Yt,spotify:track:1tRxwf8Q0AcshfHuaD86Yt,https://api.spotify.com/v1/tracks/1tRxwf8Q0Acs...,https://api.spotify.com/v1/audio-analysis/1tRx...,236456,4


In [305]:
# Option_2
# df_hot_song = pd.merge(left=df_titles, right=df_features_hot_song, how="ïnner", on="id").drop(columns="ïd_x",axis=1)

## Once the previous function has been created, create another function add_audio_features(df, audio_features_df) to concat a given dataframe with the dataframe containing the audio features alongside any other desired info, and return the extended data frame.

In [300]:
#in this function we as well drop the column Unnamed and id column, because both df have id columns.
def add_audio_features(df, audio_features_df):
    
    df4=df.copy()
    df5=audio_features_df.copy()
    df4=df4.drop(columns = ['Unnamed: 0',"id"], axis=1).reset_index(drop=True)
    df5=df5.reset_index(drop=True)
    df_final = pd.concat([df4, df5], axis=1)
    return df_final


In [303]:
df_hot_songs_final=add_audio_features(df_titles, df_features_hot_song)
df_hot_songs_final

,title,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Anti-Hero,Taylor Swift,0.637,0.643,4,-6.571,1,0.0519,0.13000,0.000002,0.1420,0.5330,97.008,audio_features,0V3wPSX9ygBnCm8psDIegu,spotify:track:0V3wPSX9ygBnCm8psDIegu,https://api.spotify.com/v1/tracks/0V3wPSX9ygBn...,https://api.spotify.com/v1/audio-analysis/0V3w...,200690,4
1,Lift Me Up,Rihanna,0.247,0.299,9,-6.083,1,0.0315,0.89900,0.000000,0.1310,0.1720,177.115,audio_features,35ovElsgyAtQwYPYnZJECg,spotify:track:35ovElsgyAtQwYPYnZJECg,https://api.spotify.com/v1/tracks/35ovElsgyAtQ...,https://api.spotify.com/v1/audio-analysis/35ov...,196520,4
2,Bad Habit,Steve Lacy,0.686,0.507,1,-7.097,1,0.0357,0.62600,0.000075,0.3830,0.6900,168.955,audio_features,5CM4UuQ9Gnd6K2YyKGPMoK,spotify:track:5CM4UuQ9Gnd6K2YyKGPMoK,https://api.spotify.com/v1/tracks/5CM4UuQ9Gnd6...,https://api.spotify.com/v1/audio-analysis/5CM4...,232115,4
3,As It Was,Harry Styles,0.520,0.731,6,-5.338,0,0.0557,0.34200,0.001010,0.3110,0.6620,173.930,audio_features,4LRPiXqCikLlN15c3yImP7,spotify:track:4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303,4
4,Lavender Haze,Taylor Swift,0.733,0.436,10,-10.489,1,0.0800,0.25800,0.000573,0.1570,0.0976,96.985,audio_features,5jQI2r1RdgtuT8S3iG8zFC,spotify:track:5jQI2r1RdgtuT8S3iG8zFC,https://api.spotify.com/v1/tracks/5jQI2r1Rdgtu...,https://api.spotify.com/v1/audio-analysis/5jQI...,202396,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,Betty (Get Money),Yung Gravy,0.723,0.772,1,-6.784,1,0.0792,0.00524,0.000012,0.3670,0.5800,102.004,audio_features,4cacyP5c3PMlfnyjpg13xW,spotify:track:4cacyP5c3PMlfnyjpg13xW,https://api.spotify.com/v1/tracks/4cacyP5c3PMl...,https://api.spotify.com/v1/audio-analysis/4cac...,146471,4
71,300 Blackout,Kodak Black,0.669,0.741,0,-6.545,0,0.3160,0.01980,0.000000,0.2230,0.3960,167.903,audio_features,3y3NwQPQAjKCunRz2eKggA,spotify:track:3y3NwQPQAjKCunRz2eKggA,https://api.spotify.com/v1/tracks/3y3NwQPQAjKC...,https://api.spotify.com/v1/audio-analysis/3y3N...,166494,4
72,Dear Reader,Taylor Swift,0.562,0.388,0,-12.088,1,0.0705,0.48100,0.001450,0.1170,0.1590,107.747,audio_features,3QF5RsWzK1lCvf2o2cY65P,spotify:track:3QF5RsWzK1lCvf2o2cY65P,https://api.spotify.com/v1/tracks/3QF5RsWzK1lC...,https://api.spotify.com/v1/audio-analysis/3QF5...,225194,4
73,Country On,Luke Bryan,0.520,0.751,5,-5.064,1,0.0551,0.17100,0.000007,0.0624,0.5190,156.044,audio_features,1tRxwf8Q0AcshfHuaD86Yt,spotify:track:1tRxwf8Q0AcshfHuaD86Yt,https://api.spotify.com/v1/tracks/1tRxwf8Q0Acs...,https://api.spotify.com/v1/audio-analysis/1tRx...,236456,4


In [306]:
#Let's call all the functions for non_hot_songs
result  = asking_id(nothotsongs)
result = result.loc[~result['id'].str.contains('None')]


Chunk:  0
Searching for song:  Like a Rolling Stone from artist:  Bob Dylan
Searching for song:  (I Can't Get No) Satisfaction' from artist:  The Rolling Stones
Searching for song:  Imagine from artist:  John Lennon
Searching for song:  What's Going On from artist:  Marvin Gaye
Searching for song:  Respect from artist:  Aretha Franklin
Searching for song:  Good Vibrations from artist:  The Beach Boys
Searching for song:  Johnny B. Goode from artist:  Chuck Berry
Searching for song:  Hey Jude from artist:  The Beatles
Searching for song:  Smells Like Teen Spirit from artist:  Nirvana
Searching for song:  What'd I Say from artist:  Ray Charles
Song not found!
Searching for song:  My Generation from artist:  The Who
Searching for song:  A Change Is Gonna Come from artist:  Sam Cooke
Searching for song:  Yesterday from artist:  The Beatles
Searching for song:  Blowin' in the Wind from artist:  Bob Dylan
Searching for song:  London Calling from artist:  The Clash
Searching for song:  I Want

Searching for song:  Mystery Train from artist:  Elvis Presley
Searching for song:  Strawberry Fields Forever from artist:  The Beatles
Searching for song:  Whole Lotta Love from artist:  Led Zeppelin
Searching for song:  Summertime Blues from artist:  Eddie Cochran
Searching for song:  Superstition from artist:  Stevie Wonder
Searching for song:  California Girls from artist:  The Beach Boys
Searching for song:  Papas Got A Brand New Bag from artist:  James Brown
Searching for song:  Walk On By from artist:  Dionne Warwick
Searching for song:  Crying from artist:  Roy Orbison
Searching for song:  Tangled Up in Blue from artist:  Bob Dylan
Searching for song:  Jailhouse Rock from artist:  Elvis Presley
Searching for song:  Redemption Song from artist:  Bob Marley and the Wailers
Searching for song:  Sunshine of Your Love from artist:  Cream
Searching for song:  She Loves You from artist:  The Beatles
Searching for song:  For What Its Worth from artist:  Buffalo Springfield
Searching fo

Searching for song:  Loser from artist:  Beck
Searching for song:  Flash Light from artist:  Parliament
Searching for song:  Hey Joe from artist:  The Jimi Hendrix Experience

Waiting 10 seconds before the next chunk
Chunk:  250
Searching for song:  Black Dog from artist:  Led Zeppelin
Searching for song:  Tired of Being Alone from artist:  Al Green
Searching for song:  Train in Vain from artist:  The Clash
Searching for song:  Shes Not There from artist:  The Zombies
Searching for song:  Stan from artist:  Eminem feat. Dido
Searching for song:  Cant Buy Me Love from artist:  The Beatles
Searching for song:  Money (Thats What I Want) from artist:  Barrett Strong
Searching for song:  Walk This Way from artist:  Run-DMC
Song not found!
Searching for song:  Summer Babe (Winter Version) from artist:  Pavement
Searching for song:  Smokestack Lightning from artist:  Howlin' Wolf
Searching for song:  (Whats So Funny About) Peace from artist:  Elvis Costello and the Attractions
Song not found!

Searching for song:  I Cant Explain from artist:  The Who
Searching for song:  The Wind Cries Mary from artist:  The Jimi Hendrix Experience
Song not found!
Searching for song:  Im a Man from artist:  Bo Diddley
Searching for song:  Personal Jesus from artist:  Depeche Mode
Searching for song:  White Room from artist:  Cream
Searching for song:  How Deep Is Your Love from artist:  Bee Gees
Searching for song:  Unchained Melody from artist:  The Righteous Brothers
Searching for song:  Highway 61 Revisited from artist:  Bob Dylan
Searching for song:  The Letter from artist:  The Box Tops
Searching for song:  Complete Control from artist:  The Clash
Searching for song:  All You Need is Love from artist:  The Beatles
Searching for song:  Killing Me Softly With His Song from artist:  Roberta Flack
Searching for song:  Got My Mojo Working from artist:  Muddy Waters
Searching for song:  Nowhere to Run from artist:  Martha and the Vandellas
Searching for song:  Little Wing from artist:  The Ji

Searching for song:  The Twist from artist:  Chubby Checker
Searching for song:  Penny Lane from artist:  The Beatles
Searching for song:  Heroin from artist:  The Velvet Underground
Searching for song:  Leader of the Pack from artist:  The Shangri-Las
Searching for song:  Pressure Drop from artist:  Toots and the Maytals
Searching for song:  Come As You Are from artist:  Nirvana
Searching for song:  I Got You Babe from artist:  Sonny and Cher

Waiting 10 seconds before the next chunk


In [307]:
df_titles_not_hot, df_features_not_hot_song=asking_features(result)

Chunk:  0
Waiting 10 seconds before the next chunk
Chunk:  50
Waiting 10 seconds before the next chunk
Chunk:  100
Waiting 10 seconds before the next chunk
Chunk:  150
Waiting 10 seconds before the next chunk
Chunk:  200
Waiting 10 seconds before the next chunk
Chunk:  250
Waiting 10 seconds before the next chunk
Chunk:  300
Waiting 10 seconds before the next chunk
Chunk:  350
Waiting 10 seconds before the next chunk
Chunk:  400
Waiting 10 seconds before the next chunk
Chunk:  450
Waiting 10 seconds before the next chunk


In [308]:
df_not_hot_songs_final=add_audio_features(df_titles_not_hot, df_features_not_hot_song)
df_not_hot_songs_final

,title,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Like a Rolling Stone,Bob Dylan,0.482,0.721,0,-6.839,1,0.0321,0.73100,0.000000,0.1890,0.557,95.263,audio_features,3AhXZa8sUQht0UEdBJgpGc,spotify:track:3AhXZa8sUQht0UEdBJgpGc,https://api.spotify.com/v1/tracks/3AhXZa8sUQht...,https://api.spotify.com/v1/audio-analysis/3AhX...,369600,4
1,(I Can't Get No) Satisfaction',The Rolling Stones,0.723,0.863,2,-7.890,1,0.0338,0.03830,0.031700,0.1280,0.931,136.302,audio_features,2PzU4IB8Dr6mxV3lHuaG34,spotify:track:2PzU4IB8Dr6mxV3lHuaG34,https://api.spotify.com/v1/tracks/2PzU4IB8Dr6m...,https://api.spotify.com/v1/audio-analysis/2PzU...,222813,4
2,Imagine,John Lennon,0.547,0.257,0,-12.358,1,0.0252,0.90700,0.183000,0.0935,0.169,75.752,audio_features,7pKfPomDEeI4TPT6EOYjn9,spotify:track:7pKfPomDEeI4TPT6EOYjn9,https://api.spotify.com/v1/tracks/7pKfPomDEeI4...,https://api.spotify.com/v1/audio-analysis/7pKf...,187867,4
3,What's Going On,Marvin Gaye,0.283,0.716,1,-9.632,0,0.0986,0.44700,0.000000,0.3990,0.828,201.960,audio_features,3Um9toULmYFGCpvaIPFw7l,spotify:track:3Um9toULmYFGCpvaIPFw7l,https://api.spotify.com/v1/tracks/3Um9toULmYFG...,https://api.spotify.com/v1/audio-analysis/3Um9...,233000,4
4,Respect,Aretha Franklin,0.805,0.558,0,-5.226,1,0.0410,0.16400,0.000022,0.0546,0.965,114.950,audio_features,7s25THrKz86DM225dOYwnr,spotify:track:7s25THrKz86DM225dOYwnr,https://api.spotify.com/v1/tracks/7s25THrKz86D...,https://api.spotify.com/v1/audio-analysis/7s25...,147600,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,Heroin,The Velvet Underground,0.201,0.777,1,-10.167,1,0.0705,0.30400,0.780000,0.0981,0.235,145.733,audio_features,5by3w3NXvwDpV9FBSOR35u,spotify:track:5by3w3NXvwDpV9FBSOR35u,https://api.spotify.com/v1/tracks/5by3w3NXvwDp...,https://api.spotify.com/v1/audio-analysis/5by3...,433573,4
481,Leader of the Pack,The Shangri-Las,0.417,0.546,0,-8.710,1,0.0858,0.72500,0.000000,0.5430,0.310,126.224,audio_features,6wzLLGFlWQ5jqTL13MU069,spotify:track:6wzLLGFlWQ5jqTL13MU069,https://api.spotify.com/v1/tracks/6wzLLGFlWQ5j...,https://api.spotify.com/v1/audio-analysis/6wzL...,173533,4
482,Pressure Drop,Toots and the Maytals,0.653,0.459,8,-12.180,1,0.0369,0.60300,0.000000,0.0536,0.986,146.296,audio_features,2EISedV2npKMtvygkykV2r,spotify:track:2EISedV2npKMtvygkykV2r,https://api.spotify.com/v1/tracks/2EISedV2npKM...,https://api.spotify.com/v1/audio-analysis/2EIS...,176693,4
483,Come As You Are,Nirvana,0.500,0.824,4,-5.846,0,0.0388,0.00016,0.001610,0.0916,0.539,120.125,audio_features,4P5KoWXOxwuobLmHXLMobV,spotify:track:4P5KoWXOxwuobLmHXLMobV,https://api.spotify.com/v1/tracks/4P5KoWXOxwuo...,https://api.spotify.com/v1/audio-analysis/4P5K...,218920,4


In [309]:
#I have now 2 data frames: df_hot_songs_final and df_not_hot_songs_final
#Let's add the column with letter H for hot songs and N for not hot songs
df_hot_songs_final['set']="H"
df_not_hot_songs_final['set']='N'

In [312]:
#making a csv files
df_hot_songs_final.to_csv("hot_songs_final.csv")
df_not_hot_songs_final.to_csv("not_hot_songs_final.csv")

